In [43]:
import torch

import numpy as np
import scipy as sp
import math
import copy

from dataclasses import dataclass

In [44]:
def nesterov(x_0, grad, L, mu, K):
    x_cur = x_0
    y_cur = x_0
    x_list = [x_0]
    y_list = [x_0]
    for i in range(K):
        x_upd = y_cur - (1/L)*(grad(y_cur))
        y_upd = x_upd + ((np.sqrt(L) - np.sqrt(mu))/(np.sqrt(L) + np.sqrt(mu))) * (x_upd - x_cur)


        x_list.append(x_upd)
        y_list.append(y_upd)

        x_cur = x_upd
        y_cur = y_upd

    return x_list, y_list

In [50]:
def grad_quad(x):
    A = np.array([[L,0],[0,1]])
    b = np.array([2,1])
    return A@x+b

# hyperparameters
@dataclass
class CFG:
    x_0 = np.array([3, 1])
    L = 100
    K = 100000
    mu = 1

    x_list_nes, y_list_nes = nesterov(x_0, grad_quad, L, mu, K)

    L_x = 10
    L_y = 1000
    mu_x = 0.1
    mu_y = 0.1
    A = np.array([[L_x, 0, 0, 0], [0, mu_x, 0, 0], [0, 0, L_y, 0], [0, 0, 0, mu_y]])
    b = np.array([2, 1, 3, 1])

    x_star = x_list_nes[-1]
    x_star = np.linalg.solve(A, -b)
    x_0 = np.array([1, 2])
    y_0 = np.array([2, 3])
    n = 4
    x_init = np.array(list(x_0) + list(y_0))

    d = 1 / 2 * (x_init.T @ A @ x_init + b.T @ x_init)
    theta = np.linalg.norm(x_init - x_star)
    alpha = 0.1
    tau = 1 / (alpha * n * n + 1)
    x_0 = np.array([3, 1])


def grad_quad_4(x):
    A = np.array([[CFG.L_x,0,0,0],[0,CFG.mu_x,0,0],[0,0,CFG.L_y,0],[0,0,0,CFG.mu_y]])
    b = np.array([2,1,3,1])
    return A@x+b

# Grads
def grad_x(x,y):
    vec = np.array(list(x)+list(y))
    res = CFG.A@vec+CFG.b
    return res[0:2]


def grad_y(x,y):
    vec = np.array(list(x)+list(y))
    res = CFG.A@vec+CFG.b
    return res[2:4]

In [ ]:
# TODO: We need proection method because we have optimisation with limitations.

In [46]:
# ACRCD


def ACRCD(alpha, tau, theta, x_0, y_0, d, K):
    x_list = [x_0]
    y_list = [y_0]

    x_cur = x_0
    y_cur = y_0

    z_cur_x = x_0
    z_cur_y = y_0

    q_cur_x = x_0
    q_cur_y = y_0



    for i in range(K):

        x_upd = tau*z_cur_x+(1-tau)*q_cur_x
        y_upd = tau*z_cur_y+(1-tau)*q_cur_y

        index_p = np.random.choice([0,1],p=[0.5,0.5])
        if index_p == 0:
            q_upd_x = x_upd - (1/CFG.L_x)*grad_x(x_upd, y_upd)
            q_upd_y = q_cur_y

        if index_p == 1:
            q_upd_y = y_upd - (1/CFG.L_y)*grad_y(x_upd, y_upd)
            q_upd_x = q_cur_x

        if index_p == 0:
            z_upd_x = z_cur_x - (1/CFG.L_x)*alpha*CFG.n*grad_x(x_upd,y_upd)
            z_upd_y = z_cur_y

        if index_p == 1:
            z_upd_y = z_cur_y - (1/CFG.L_y)*alpha*CFG.n*grad_y(x_upd,y_upd)
            z_upd_x = z_cur_x



        x_list.append(x_upd)
        y_list.append(y_upd)

        x_cur = x_upd
        y_cur = y_upd

        z_cur_x = z_upd_x
        z_cur_y = z_upd_y

        q_cur_x = q_upd_x
        q_cur_y = q_upd_y


    return x_list, y_list


x_list_ACRCD, y_list_ACRCD = ACRCD(CFG.alpha, CFG.tau, CFG.theta, CFG.x_0, CFG.y_0, CFG.d, 1*CFG.K)

In [47]:
CFG.alpha

0.1

In [48]:
x_list_ACRCD[-1]

array([ -0.2, -10. ])

In [49]:
y_list_ACRCD[-1]

array([-3.00000000e-03, -8.23129975e+00])